## Restaurant NER (AE - Aspect Extraction)

### Preprocess training data

In [3]:
import json
import pandas as pd

rest_ner_training_file = '/Users/sean.huang/Documents/my/books and courses/ztgg/Shared Code/BERT-for-ABSA/ae/rest/train.json'
rest_ner_training_samples = json.load(open(rest_ner_training_file))
len(rest_ner_training_samples)

1850

In [9]:
rest_ner_training_samples['0']

{'label': ['O', 'O', 'O', 'B'], 'sentence': ['I', 'LOVE', 'their', 'Thai']}

In [10]:
dict_rest_ner_training_samples = {'sentence_id':[], 'sentence':[], 'label':[]}

for key in rest_ner_training_samples.keys():
    dict_rest_ner_training_samples['sentence_id'].append(key)
    dict_rest_ner_training_samples['sentence'].append(rest_ner_training_samples[key]['sentence'])
    dict_rest_ner_training_samples['label'].append(rest_ner_training_samples[key]['label'])
    
dict_rest_ner_training_samples

df_rest_ner_training_samples = pd.DataFrame.from_dict(dict_rest_ner_training_samples)
df_rest_ner_training_samples

,sentence_id,sentence,label
0,0,"[I, LOVE, their, Thai]","[O, O, O, B]"
1,1,"[The, service, was, attentive, .]","[O, B, O, O, O]"
2,2,"[I, go, twice, a, month, !]","[O, O, O, O, O, O]"
3,3,"[The, food, was, average, to, above, -, averag...","[O, B, O, O, O, O, O, O, O, O, B, I, I, O, O, ..."
4,4,"[Ask, for, Usha, ,, the, nicest, bartender, in...","[O, O, B, O, O, O, O, O, O, O]"
...,...,...,...
1845,1845,"[You, can, get, a, table, without, a, reservat...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1846,1846,"[I, have, eaten, at, some, of, the, 'best, ', ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1847,1847,"[Worth, visiting, the, 1st, Ave, spot, because...","[O, O, O, B, I, I, O, O, O, O, O, O, O]"
1848,1848,"[Its, located, in, greenewich, village, .]","[O, O, O, O, O, O]"


In [11]:
df_rest_ner_training_samples_exploded = df_rest_ner_training_samples.explode(['sentence', 'label'])
df_rest_ner_training_samples_exploded

,sentence_id,sentence,label
0,0,I,O
0,0,LOVE,O
0,0,their,O
0,0,Thai,B
1,1,The,O
...,...,...,...
1849,1849,front,O
1849,1849,of,O
1849,1849,the,O
1849,1849,guest,O


In [12]:
def get_mapping(data, column):
    t2id = {}
    id2t = {}
 
    vocab = list(set(data[column].to_list()))
    
    id2t = {i:t for i, t in enumerate(vocab)}
    t2id = {t:i for  i, t in enumerate(vocab)}
    return t2id, id2t


token2id, id2token = get_mapping(df_rest_ner_training_samples_exploded, 'sentence')
tag2id, id2tag = get_mapping(df_rest_ner_training_samples_exploded, 'label')

print(list(token2id.items())[0:5], list(id2token.items())[0:5])
print(tag2id, id2tag)

[('Dumbfoundingly', 0), ('as', 1), ('fix', 2), ('Yamato', 3), ('extra', 4)] [(0, 'Dumbfoundingly'), (1, 'as'), (2, 'fix'), (3, 'Yamato'), (4, 'extra')]
{'I': 0, 'O': 1, 'B': 2} {0: 'I', 1: 'O', 2: 'B'}


In [13]:
df_rest_ner_training_samples_exploded['word_id'] = df_rest_ner_training_samples_exploded['sentence'].map(token2id)
df_rest_ner_training_samples_exploded['tag_id'] = df_rest_ner_training_samples_exploded['label'].map(tag2id)
df_rest_ner_training_samples_exploded.head(20)

,sentence_id,sentence,label,word_id,tag_id
0,0,I,O,496,1
0,0,LOVE,O,2564,1
0,0,their,O,1248,1
0,0,Thai,B,296,2
1,1,The,O,1194,1
1,1,service,B,3339,2
1,1,was,O,2107,1
1,1,attentive,O,1797,1
1,1,.,O,2193,1
2,2,I,O,496,1


In [20]:
df_rest_ner_training_samples_exploded.isnull().sum()

sentence    0
label       0
word_id     0
tag_id      0
dtype: int64

In [176]:
df_rest_ner_training_samples = df_rest_ner_training_samples_exploded.groupby(['sentence_id'])['sentence', 'label', 'word_id', 'tag_id'].agg(lambda x: list(x))
df_rest_ner_training_samples

/var/folders/57/qcnnk6sn1vvbtwg1g1xc6kmw0000gp/T/ipykernel_47302/3037838837.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_rest_ner_training_samples = df_rest_ner_training_samples_exploded.groupby(['sentence_id'])['sentence', 'label', 'word_id', 'tag_id'].agg(lambda x: list(x))


,sentence,label,word_id,tag_id
sentence_id,,,,
0,"[I, LOVE, their, Thai]","[O, O, O, B]","[529, 3167, 1628, 2790]","[2, 2, 2, 1]"
1,"[The, service, was, attentive, .]","[O, B, O, O, O]","[2898, 115, 2073, 554, 2835]","[2, 1, 2, 2, 2]"
10,"[Everything, is, always, cooked, to, perfectio...","[O, O, O, O, O, O, O, O, B, O, O, O, O, B, O, ...","[313, 109, 2381, 2296, 3308, 2946, 3509, 1261,...","[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, ..."
100,"[good, music, ,, great, food, ,, speedy, servi...","[O, B, O, O, B, O, O, B, O, O, O]","[687, 1927, 3509, 3143, 725, 3509, 903, 115, 7...","[2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2]"
1000,"[But, the, pizza, is, way, to, expensive, .]","[O, O, B, O, O, O, O, O]","[891, 1261, 1812, 109, 1749, 3308, 32, 2835]","[2, 2, 1, 2, 2, 2, 2, 2]"
...,...,...,...,...
995,"[The, only, beverage, we, did, receive, was, w...","[O, O, O, O, O, O, O, O, O, O, O, O]","[2898, 899, 479, 3270, 3263, 2692, 2073, 466, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
996,"[A, friend, from, Seattle, and, I, went, on, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[3077, 1371, 2882, 739, 3425, 529, 1245, 2952,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
997,"[Went, on, a, 3, day, oyster, binge, ,, with, ...","[O, O, O, O, O, O, O, O, O, B, O, O, O, O, O, ...","[2427, 2952, 2363, 3047, 3312, 401, 1111, 3509...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, ..."


In [177]:
max_sent_length = max([len(each) for each in list(df_rest_ner_training_samples['sentence'])])
max_sent_length

74

### Load and finetune pretrained Bert model

In [178]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

pretrained_bert_model_name = 'bert-base-uncased'
pretrained_bert_tokenizer = AutoTokenizer.from_pretrained(pretrained_bert_model_name)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/sean.huang/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncase

In [179]:
tokens = pretrained_bert_tokenizer(list(df_rest_ner_training_samples['sentence'])[0], is_split_into_words=True)
tokens

{'input_ids': [101, 1045, 2293, 2037, 7273, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [181]:
from datasets import Dataset

def align_ner_labels(samples):
    inputs = pretrained_bert_tokenizer(samples['sentence'], truncation=True, padding='max_length', max_length=128, is_split_into_words=True)

    label_input = []
    for i, label in enumerate(samples['label']):
        word_ids = inputs.word_ids(batch_index=i)
        
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(tag2id[label[word_idx]])
        
        if i == 0:
            print(i)
            print(label)
            print(word_ids)
            print(label_ids)
            
        label_input.append(label_ids)
        
    inputs['labels'] = label_input
    
    # print('inputs', inputs.keys())
    
    return inputs

train_datasets = Dataset.from_pandas(df_rest_ner_training_samples[:1600]).map(align_ner_labels, batched=True, batch_size=100).remove_columns(['sentence', 'label', 'word_id', 'tag_id', 'sentence_id'])
validation_datasets = Dataset.from_pandas(df_rest_ner_training_samples[1600:]).map(align_ner_labels, batched=True, batch_size=10).remove_columns(['sentence', 'label', 'word_id', 'tag_id', 'sentence_id'])

  0%|          | 0/16 [00:00<?, ?ba/s]

0
['O', 'O', 'O', 'B']
[None, 0, 1, 2, 3, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[-100, 2, 2, 2, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
0
['O', 'O', 'O', 'O', 'O']
[None, 0, 1, 2, 3, 4, N

  0%|          | 0/25 [00:00<?, ?ba/s]

0
['B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[None, 0, 0, 1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 9, None, None, None, None, None, None, None, None, None, None, None]
[-100, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
0
['O', 'B', 'O', 'O', 'O', 'O']
[None, 0, 1, 2, 3, 4, 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[-100, 2, 1, 2, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
0
['B', 'O', 'O', 'O', 'B', 'O', 'O', 'O']
[None, 0, 1, 2, 3, 4, 5, 6, 7, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[-100, 1, 2, 2, 2, 1, 2, 2, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
0
['O', 'B', 'I', 'O', 'O', 'O'

In [182]:
train_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1600
})

In [72]:
validation_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 250
})

In [59]:
import torch 

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')   # mps is for Apple M1 GPU
device

device(type='mps')

In [60]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch

pretrained_bert_model = AutoModelForTokenClassification.from_pretrained(pretrained_bert_model_name, num_labels=len(tag2id.keys()), id2label=id2tag, label2id=tag2id)

pretrained_bert_model.to(device)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/sean.huang/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "I",
    "1": "B",
    "2": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B": 1,
    "I": 0,
    "O": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "t

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [75]:
args = TrainingArguments(
    output_dir='./rest_ner_model',
    per_device_train_batch_size=10,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    report_to='none'
)

data_collator = DataCollatorForTokenClassification(pretrained_bert_tokenizer)
    
trainer = Trainer(
    model=pretrained_bert_model,
    args=args,
    train_dataset=train_datasets,
    eval_dataset=validation_datasets,
    # data_collator=data_collator, # for padding, optional here
    tokenizer=pretrained_bert_tokenizer
)

trainer.train()
trainer.save_model()


# solution to "AttributeError: module 'torch.distributed' has no attribute 'is_initialized'"
# https://stackoverflow.com/questions/72641886/attributeerror-module-torch-distributed-has-no-attribute-is-initialized-in/72641887

PyTorch: setting up devices
torch.distributed process group is initialized, but local_rank == -1. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
/Users/sean.huang/miniconda3/envs/m1_new/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 1600


Epoch,Training Loss,Validation Loss
1,0.007100,0.218634
2,0.006500,0.095442
3,0.002000,0.083480
4,0.003500,0.103103
5,0.006900,0.100134
6,0.000900,0.111565
7,0.001200,0.112978
8,0.000300,0.118741
9,0.000500,0.128856
10,0.000300,0.127052


***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
Saving model checkpoint to ./rest_ner_model/checkpoint-500
Configuration saved in ./rest_ner_model/checkpoint-500/config.json
Model weights saved in ./rest_ner_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./rest_ner_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./rest_ner_model/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
Saving model checkpoint to ./rest_ner_model/checkpoint-1000
Configuration saved in ./rest_ner_model/checkpoint-1000/config.json
Model weights saved in ./rest_ner_model/checkpoint-1000/pytorch_model.bin
tokenizer confi

In [84]:
from transformers import EarlyStoppingCallback


args = TrainingArguments(
    output_dir='./rest_ner_model',
    per_device_train_batch_size=10,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    logging_strategy='steps',
    evaluation_strategy='steps',
    eval_steps = 100,
    load_best_model_at_end=True,
    metric_for_best_model=eval_loss,
    report_to='none'
)

data_collator = DataCollatorForTokenClassification(pretrained_bert_tokenizer)
    
trainer = Trainer(
    model=pretrained_bert_model,
    args=args,
    train_dataset=train_datasets,
    eval_dataset=validation_datasets,
    # data_collator=data_collator, # for padding, optional here
    tokenizer=pretrained_bert_tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.save_model()


PyTorch: setting up devices
torch.distributed process group is initialized, but local_rank == -1. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
/Users/sean.huang/miniconda3/envs/m1_new/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 1600


Step,Training Loss,Validation Loss
100,No log,0.190032
200,0.002100,0.168284
300,0.002100,0.127641
400,0.005700,0.129792
500,0.005300,0.149050
600,0.005300,0.124293
700,0.003800,0.102385
800,0.003800,0.097123
900,0.001700,0.099075
1000,0.001500,0.108774


***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
Saving model checkpoint to ./rest_ner_model/checkpoint-500
Configuration saved in ./rest_ner_model/checkpoint-500/config.json
Model weights saved in ./rest_ner_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./rest_ner_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./rest_ner_model/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Running Evaluation *****
  Num examples = 250
  Batch size = 2
***** Runn

### Load finetune Bert model for Rest NER task, and do prediction

In [14]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

finetuned_rest_tokenizer = AutoTokenizer.from_pretrained('./rest_ner_model')
finetuned_rest_model = AutoModelForTokenClassification.from_pretrained('./rest_ner_model', num_labels=len(tag2id.keys()))

In [15]:
from transformers import pipeline

rest_ner_pipeline = pipeline('ner', model=finetuned_rest_model, tokenizer=finetuned_rest_tokenizer)

examples = ['Serves really good food, nice environment', 'Oh yeah ... the view was good too']
rest_ner_results = rest_ner_pipeline(examples)
print(rest_ner_results)

[[{'entity': 'B', 'score': 0.9999516, 'index': 4, 'word': 'food', 'start': 19, 'end': 23}, {'entity': 'B', 'score': 0.999948, 'index': 7, 'word': 'environment', 'start': 30, 'end': 41}], [{'entity': 'B', 'score': 0.999959, 'index': 7, 'word': 'view', 'start': 16, 'end': 20}]]


### Evaluation

In [4]:
rest_ner_test_file = '/Users/sean.huang/Documents/my/books and courses/ztgg/Shared Code/BERT-for-ABSA/ae/rest/test.json'
rest_ner_test_samples = json.load(open(rest_ner_test_file))
len(rest_ner_test_samples)

676

In [16]:
dict_rest_ner_test_samples = {'sentence_id':[], 'sentence':[], 'full_sentence':[], 'label':[]}

for key in rest_ner_test_samples.keys():
    dict_rest_ner_test_samples['sentence_id'].append(key)
    dict_rest_ner_test_samples['sentence'].append(rest_ner_test_samples[key]['sentence'])
    dict_rest_ner_test_samples['full_sentence'].append(' '.join(rest_ner_test_samples[key]['sentence']))
    dict_rest_ner_test_samples['label'].append(rest_ner_test_samples[key]['label'])
    

In [17]:
rest_ner_results = rest_ner_pipeline(dict_rest_ner_test_samples['full_sentence'])

In [18]:
rest_ner_results[:5]

[[],
 [{'entity': 'B',
   'score': 0.9999312,
   'index': 4,
   'word': 'su',
   'start': 19,
   'end': 21},
  {'entity': 'B',
   'score': 0.9999331,
   'index': 5,
   'word': '##shi',
   'start': 21,
   'end': 24}],
 [{'entity': 'B',
   'score': 0.99994266,
   'index': 4,
   'word': 'portions',
   'start': 16,
   'end': 24}],
 [{'entity': 'B',
   'score': 0.9995384,
   'index': 1,
   'word': 'green',
   'start': 0,
   'end': 5},
  {'entity': 'I',
   'score': 0.99952495,
   'index': 2,
   'word': 'tea',
   'start': 6,
   'end': 9},
  {'entity': 'I',
   'score': 0.999559,
   'index': 3,
   'word': 'cr',
   'start': 10,
   'end': 12},
  {'entity': 'I',
   'score': 0.99959844,
   'index': 4,
   'word': '##eme',
   'start': 12,
   'end': 15},
  {'entity': 'I',
   'score': 0.999647,
   'index': 5,
   'word': 'br',
   'start': 16,
   'end': 18},
  {'entity': 'I',
   'score': 0.99965715,
   'index': 6,
   'word': '##ule',
   'start': 18,
   'end': 21},
  {'entity': 'I',
   'score': 0.9996382,

In [19]:
reformatted_rest_ner_results = []

for sentence_result in rest_ner_results:
    if len(sentence_result) == 0:
        reformatted_rest_ner_results.append({})
        continue
    last_label = sentence_result[0]['entity']
    last_token = sentence_result[0]['word']
    reformatted_sent_result = {}
    for token_result in sentence_result[1:]:
        label = token_result['entity']
        token = token_result['word']
        
        if token.startswith('##') is False:
            if last_label != '':
                reformatted_sent_result[last_token] = last_label
            last_token = token
            last_label = label
        else:
            last_token = last_token + token[2:] # remove '##' 
            last_label = label            

    if last_label != '':
        reformatted_sent_result[last_token] = last_label
    
    reformatted_rest_ner_results.append(reformatted_sent_result)
    
reformatted_rest_ner_results[:5]         

[{},
 {'sushi': 'B'},
 {'portions': 'B'},
 {'green': 'B', 'tea': 'I', 'creme': 'I', 'brulee': 'I'},
 {}]

In [20]:
y_pred = []

for tokens, ner_tags in zip(dict_rest_ner_test_samples['sentence'], reformatted_rest_ner_results):
    
    pred = []
    for token in tokens:
        if token not in ner_tags.keys():
            pred.append('O')
            y_pred.append('O')
        else:
            pred.append(ner_tags[token])
            y_pred.append(ner_tags[token])
    # print(tokens, ner_tags, pred)
    
y_pred[:10]

['O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O']

In [21]:
len(y_pred)

10096

In [22]:
y_true = []
for each in dict_rest_ner_test_samples['label']:
    y_true.extend(each)

len(y_true)

10096

In [23]:
from sklearn import metrics
metrics.accuracy_score(y_true, y_pred)

0.9539421553090333

In [24]:
metrics.precision_recall_fscore_support(y_true, y_pred, labels=['B', 'I', 'O'])

(array([0.79467681, 0.73205742, 0.96784532]),
 array([0.68300654, 0.54642857, 0.98435463]),
 array([0.73462214, 0.62576687, 0.97603016]),
 array([ 612,  280, 9204]))

In [25]:
metrics.confusion_matrix(y_true, y_pred, labels=['B', 'I', 'O'])

array([[ 418,    9,  185],
       [  11,  153,  116],
       [  97,   47, 9060]])

## Home Work: do laptop NER (AE)

...